In [3]:
import os
import xarray as xr

path = rf"E:\client_v2_data"

files = os.listdir(path)
files = [f for f in files if f.endswith('.tif')]

In [4]:
WB_country = xr.open_dataset(rf"{path}\WB_country_grid.nc")

minx = WB_country.x.min()
maxx = WB_country.x.max()
miny = WB_country.y.min()
maxy = WB_country.y.max()

c:\Users\ofici\AppData\Local\Programs\Python\Python311\Lib\site-packages\xarray\backends\plugins.py:80: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)
c:\Users\ofici\AppData\Local\Programs\Python\Python311\Lib\site-packages\cupy\_environment.py:216: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(


In [14]:
# Crop GPW according to the country grid
for file in files:
    (
        xr.open_dataarray(os.path.join(path, files[0]))
            .sel(x=slice(minx, maxx), y=slice(maxy, miny))
            .rio.to_raster(os.path.join(path, file.replace('.tif', '_proc.tif')))
    )
    print(f"Se creó {os.path.join(path, file.replace('.tif', '_proc.tif'))}")

In [5]:
# import hashlib
# import numpy as np

# # Define the function to generate MD5 hash
# def generate_md5(ADM2_CODE, ADM1_CODE):
#     return hashlib.md5((str(ADM2_CODE) + str(ADM1_CODE)).encode()).hexdigest()

# # Apply the function element-wise
# WB_country["ID"] = xr.apply_ufunc(
#     np.vectorize(generate_md5),
#     WB_country["ADM2_CODE"],
#     WB_country["ADM1_CODE"],
#     input_core_dims=[[], []],
#     output_core_dims=[[]],
#     vectorize=True,
# )

In [ ]:
df = WB_country.to_dataframe().reset_index()

# Create a unique ID for each unique combination of ADM1_CODE and ADM2_CODE
df['unique_id'] = df.groupby(['ADM1_CODE', 'ADM2_CODE']).ngroup()

# Convert the DataFrame back to a Dataset
WB_country_new = df.set_index(['x', 'y']).to_xarray()


KeyError: "None of ['dim_0'] are in the columns"

In [1]:
ds = df.set_index(["x", "y"]).to_xarray()

NameError: name 'df' is not defined

In [ ]:
WB_country.drop_vars(["str_to_hash", "ADM1_CODE", "ADM2_CODE"])#.to_netcdf(rf"{path}\WB_country_grid_proc.nc")